In [3]:
from scipy.stats import beta
from scipy.special import logsumexp

import numpy as np
import scipy

import matplotlib.pyplot as plt
from matplotlib import patches
import pandas as pd
from intervaltree import Interval, IntervalTree

import argparse
import os

plt.rcParams['pdf.use14corefonts'] = True

In [4]:
CSIZE = {'1': 249250621, '2': 243199373, '3': 198022430, '4': 191154276, '5': 180915260, '6': 171115067, '7': 159138663,
         '8': 146364022,
         '9': 141213431, '10': 135534747, '11': 135006516, '12': 133851895, '13': 115169878, '14': 107349540,
         '15': 102531392,
         '16': 90354753, '17': 81195210, '18': 78077248, '19': 59128983, '20': 63025520, '21': 48129895, '22': 51304566,
         'X': 156040895, 'Y': 57227415, '23': 156040895, '24': 57227415}
cluster_color_map = {'0': '#808080', '1': '#278C18', '2': '#67C8F3', '3': '#F88B10', '4': '#103129', '5': '#5D77FE',
                     '6': '#98161A',
                     '7': '#68ECAC', '8': '#F98E87', '9': '#371230', '10': '#535216', '11': '#F72424', '12': '#004F72',
                     '13': '#F34184', '14': '#3CB9B3', '15': '#B9B1F3', '16': '#8B2243', '17': '#B229BA',
                     '18': '#3A92E7',
                     '19': '#829F15', '20': '#A15BF3', '21': '#833D11', '22': '#F84B51', '23': '#204B20',
                     '24': '#2D6D74',
                     '25': '#FFA9C7', '26': '#37B371', '27': '#222A03', '28': '#3879A6', '29': '#AC3C0F',
                     '30': '#734CCC',
                     '31': '#153D49', '32': '#43154A', '33': '#7B5870', '34': '#576A2E', '35': '#25423A',
                     '36': '#844F3E',
                     '37': '#473A20', '38': '#3B6872', '39': '#2E6B5A', '40': '#544449', '41': '#5A617C',
                     '42': '#79424C',
                     '43': '#685D30', '44': '#314352', '45': '#475F41', '46': '#7F552C', '47': '#584F5C',
                     '48': '#DCD4C2',
                     '49': '#232224', '50': '#C8DCE0', '51': '#495145', '52': '#E0C7CE', '53': '#787F71',
                     '54': '#8E94A6',
                     '55': '#99A79C', '56': '#A28B91', '57': '#000000'}

In [5]:
basic_total_cn_dict = {'copy_neutral': 2, 'single_gain': 3, 'double_gain': 4, 'single_loss': 1, 'double_loss': 0}
basic_allelic_cn_dict = {'copy_neutral': (0, 2), 'single_gain': (1, 2), 'double_gain': (2, 2), 'single_loss': (0, 1), 'double_loss': (0, 0)}

### Get CCFs from the cluster_ccfs file

In [6]:
def get_sample_ccfs(cluster_ccfs, sample_list):
    sample_ccfs = {sample_name: {} for sample_name in sample_list}
    for (sample_name, cluster), sample_df in cluster_ccfs.groupby(['Sample_ID', 'Cluster_ID']):
        sample_ccfs[sample_name][str(cluster)] = sample_df.squeeze()['postDP_ccf_mean']

    return sample_ccfs

### Create interval trees for each chromosome for a patient

In [7]:
def make_contig_trees(acs_dfs, sample_list):
    chromosomes = pd.unique(acs_dfs[0]['Chromosome'])
    contig_tree = {str(chrom): IntervalTree() for chrom in chromosomes} # Initialize overall patient contig tree
    for seg_df, sample_name in zip(acs_dfs, sample_list): # Iterate through each sample
        for chrom, chrom_seg_df in seg_df.groupby('Chromosome'): # Iterate through each chromosome
            chrom_tree = IntervalTree( # Create sample interval trees for each chromosome for each sample of the patient
                Interval(row['Start.bp'], row['End.bp'], {sample_name: (row['mu.minor'], row['mu.major'])}) for i, row # Each interval contains seg boundaries, sample name, and copy number
                in chrom_seg_df.iterrows() if pd.notna(row['mu.minor']) and pd.notna(row['mu.major']))
            contig_tree[str(chrom)].update(chrom_tree) # Add intervals to overall patient contig tree
    # Merge interval trees for the same chromosome across samples
    for chrom in contig_tree:
        contig_tree[chrom].split_overlaps() # Split all intervals at the point where they overlap with another interval
        contig_tree[chrom].merge_equals(data_reducer=lambda o, n: dict(o, **n)) # Merge intervals with the same boundaries; the final merged information will contain copy numbers for all samples
        for interval in contig_tree[chrom].copy():
            if len(interval.data) != len(sample_list):
                contig_tree[chrom].remove(interval) # Remove any intervals that aren't covered by all samples

    return contig_tree

### Assign segs to clusters

In [9]:
def assign_segs(contig_tree, sample_ccfs, hets_dfs, purities, ploidies):
    seg_assignments = {chrom: IntervalTree() for chrom in contig_tree}
    seg_total_cn = {chrom: {} for chrom in contig_tree}
    seg_allelic_cn = {chrom: {} for chrom in contig_tree}
    seg_ccf = {chrom: {} for chrom in contig_tree}
    ccf_grid = np.arange(101) / 100
    for chrom, tree in contig_tree.items(): # Iterate through each chromosome
        for seg in tree: # Iterate through each seg
            start = seg.begin
            end = seg.end
            mu_minor = np.array([mu[0] for mu in seg.data.values()]) # Expected values of minor allele copy numbers
            mu_major = np.array([mu[1] for mu in seg.data.values()]) # Expected values of major allele copy numbers
            if all((mu_minor >= .85) & (mu_minor <= 1.15)) and all((mu_major >= .85) & (mu_major <= 1.15)): # If both copy numbers are around 1 for all samples, no cnv is called
                seg_assignments[chrom][start:end] = '0'
                seg_total_cn[chrom][(start, end)] = 2
                seg_allelic_cn[chrom][(start, end)] = (1, 1)
                continue
            classification = 'copy_neutral' # Default is CNLOH
            if any(mu_major > 1.15) and all(mu_major >= .85) and all((mu_minor <= 1.15) & (mu_minor >= .85)): # Single gain if only major allele is gained and no allele is ever lost
                classification = 'single_gain'
            if any(mu_major > 1.15) and all(mu_major >= .85) and any(mu_minor > 1.15) and all(mu_minor >= .85): # Double gain if both alleles are gained and no allele is ever lost
                classification = 'double_gain'
            if any(mu_minor < .85) and all(mu_minor <= 1.15) and all((mu_major >= .85) & (mu_major <= 1.15)): # Single loss if only minor allele is lost and no allele is ever gained
                classification = 'single_loss'
            if any(mu_minor < .85) and all(mu_minor <= 1.15) and any(mu_major < .85) and all(mu_major <= 1.15): # Double loss if both alleles are lost and no allele is ever gained
                classification = 'double_loss'
            no_hets = False
            sample_likelihoods = {}
            sample_ccf_dict = {}
            for sample, hets_df, purity, ploidy in zip(sample_ccfs, hets_dfs, purities, ploidies): # For each sample get CCFs and assignment likelihoods for every cluster
                clonal_likelihoods = {}
                # For double gains and double losses, both alleles have the same copy number so we can't use het AF information
                if classification == 'double_gain':
                    ccf = seg.data[sample][0] - 1 # CCF is copy number of double gain - 1
                    sample_ccf_dict[sample] = ccf
                    for cluster in sample_ccfs[sample]:
                        clonal_likelihoods[cluster] = np.log((1 - abs(ccf - sample_ccfs[sample][cluster])) ** 2) # Likelihood is set to 1 - mean squared error; not accurate but we're only taking the max so it doens't really matter
                elif classification == 'double_loss':
                    ccf = 1 - seg.data[sample][0] # CCF is 1 - copy number of double loss
                    sample_ccf_dict[sample] = ccf
                    for cluster in sample_ccfs[sample]:
                        clonal_likelihoods[cluster] = np.log((1 - abs(ccf - sample_ccfs[sample][cluster])) ** 2)
                # For single gains and single losses we use het info
                else:
                    hets = hets_df.loc[(hets_df['CONTIG'].astype(str) == chrom) & \
                                       hets_df['POSITION'].astype(int).between(start, end)]
                    ref_counts = hets['REF_COUNT'].values
                    alt_counts = hets['ALT_COUNT'].values
                    # Posterior distribution for true allele ratio given ref and alt counts for all hets on seg
                    posterior_het_split = np.sum(np.log(10e-25 + (
                                0.5 * beta.pdf(np.expand_dims(np.linspace(0, 1, 101), axis=1), ref_counts + 1,
                                               alt_counts + 1)) +
                                                        (0.5 * beta.pdf(np.expand_dims(np.linspace(0, 1, 101), axis=1),
                                                                        alt_counts + 1, ref_counts + 1))), axis=1)

                    posterior_het_split = posterior_het_split - scipy.special.logsumexp(posterior_het_split)
                    if hets.empty: # If no hets, don't call anything
                        no_hets = True
                        break
                    ccf_dist = np.empty(101, dtype=np.float64)
                    if classification == 'single_gain':
                        for cluster, ccf in sample_ccfs[sample].items():
                            # Calculate predicted allele ratio given cluster CCF
                            num = .5 * (1 - purity) * 2 + 2 * purity * ccf + 1 * (1 - ccf) * purity # Amount of allele 1 in normal cells + amount of allele 1 in cluster + amount of allele 1 in tumor cells but outside cluster
                            denom = (2 * (1 - purity) + purity * (ccf * 3 + (1 - ccf) * 2)) # Amount of allele 1 and allele 2 in normal cells + amount of allele 1 and allele 2 in tumor cells
                            gain = num / denom # Predicted allele ratio given CCF

                            num = .5 * (1 - purity) * 2 + 1 * purity * ccf + 1 * (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 3 + (1 - ccf) * 2))
                            gain_other = num / denom # 1 - predicted allele ratio; this is actually unnecessary since the posterior distribution is symmetrical, but it doesn't matter

                            gain = int(np.around(gain, decimals=2) * 100) # Round to the nearst hundredth
                            gain_other = int(np.around(gain_other, decimals=2) * 100)

                            lh_clone = posterior_het_split[gain] + posterior_het_split[gain_other]
                            clonal_likelihoods[cluster] = lh_clone
                        # Calculated CCF distribution
                        for i, ccf in enumerate(ccf_grid):
                            num = .5 * (1 - purity) * 2 + 2 * purity * ccf + 1 * (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 3 + (1 - ccf) * 2))
                            gain = num / denom

                            num = .5 * (1 - purity) * 2 + 1 * purity * ccf + 1 * (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 3 + (1 - ccf) * 2))
                            gain_other = num / denom

                            gain = int(np.around(gain, decimals=2) * 100)
                            gain_other = int(np.around(gain_other, decimals=2) * 100)

                            lh = np.logaddexp(posterior_het_split[gain], posterior_het_split[gain_other])
                            ccf_dist[i] = lh
                        ccf_dist = ccf_dist - scipy.special.logsumexp(ccf_dist)
                    elif classification == 'single_loss': # Similar process for single loss and CNLOH
                        for cluster, ccf in sample_ccfs[sample].items():
                            num = .5 * (1 - purity) * 2 + 0 * purity * ccf + 1 * (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 1 + (1 - ccf) * 2))
                            deletion = num / denom

                            num = .5 * (1 - purity) * 2 + 1 * purity * ccf + 1 * (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 1 + (1 - ccf) * 2))
                            deletion_other = num / denom

                            deletion = int(np.around(deletion, decimals=2) * 100)
                            deletion_other = int(np.around(deletion_other, decimals=2) * 100)

                            lh_clone = posterior_het_split[deletion] + posterior_het_split[deletion_other]
                            clonal_likelihoods[cluster] = lh_clone
                        for i, ccf in enumerate(ccf_grid):
                            num = .5 * (1 - purity) * 2 + 0 * purity * ccf + 1 * (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 1 + (1 - ccf) * 2))
                            deletion = num / denom

                            num = .5 * (1 - purity) * 2 + 1 * purity * ccf + 1 * (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 1 + (1 - ccf) * 2))
                            deletion_other = num / denom

                            deletion = int(np.around(deletion, decimals=2) * 100)
                            deletion_other = int(np.around(deletion_other, decimals=2) * 100)

                            lh = np.logaddexp(posterior_het_split[deletion], posterior_het_split[deletion_other])
                            ccf_dist[i] = lh
                        ccf_dist = ccf_dist - scipy.special.logsumexp(ccf_dist)
                    elif classification == 'copy_neutral':
                        for cluster, ccf in sample_ccfs[sample].items():
                            num = .5 * (1 - purity) * 2 + 0 * purity * ccf + (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 2 + (1 - ccf) * 2))
                            cnloh = num / denom

                            num = .5 * (1 - purity) * 2 + 2 * purity * ccf + (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 2 + (1 - ccf) * 2))
                            cnloh_other = num / denom

                            cnloh = int(np.around(cnloh, decimals=2) * 100)
                            cnloh_other = int(np.around(cnloh_other, decimals=2) * 100)

                            lh_clone = posterior_het_split[cnloh] + posterior_het_split[cnloh_other]
                            clonal_likelihoods[cluster] = lh_clone
                        for i, ccf in enumerate(ccf_grid):
                            num = .5 * (1 - purity) * 2 + 0 * purity * ccf + (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 2 + (1 - ccf) * 2))
                            cnloh = num / denom

                            num = .5 * (1 - purity) * 2 + 2 * purity * ccf + (1 - ccf) * purity
                            denom = (2 * (1 - purity) + purity * (ccf * 2 + (1 - ccf) * 2))
                            cnloh_other = num / denom

                            cnloh = int(np.around(cnloh, decimals=2) * 100)
                            cnloh_other = int(np.around(cnloh_other, decimals=2) * 100)

                            lh = np.logaddexp(posterior_het_split[cnloh], posterior_het_split[cnloh_other])
                            ccf_dist[i] = lh
                        ccf_dist = ccf_dist - scipy.special.logsumexp(ccf_dist)
                    ccf_dist = np.exp(ccf_dist)
                    sample_ccf_dict[sample] = np.sum(ccf_grid * ccf_dist)
                sample_likelihoods[sample] = clonal_likelihoods
            if no_hets: # If any of the samples has no het information, don't call the CNV
                seg_assignments[chrom][start:end] = '0'
                seg_total_cn[chrom][(start, end)] = 2
                seg_allelic_cn[chrom][(start, end)] = (1, 1)
            else:
                sample_likelihoods = pd.DataFrame(sample_likelihoods)
                sample_likelihoods -= logsumexp(sample_likelihoods, axis=0)
                sample_likelihoods_sum = sample_likelihoods.sum(axis=1) # Multiply likelihoods (adding in log space) across samples
                seg_assignments[chrom][start:end] = sample_likelihoods_sum.idxmax()
                seg_total_cn[chrom][(start, end)] = basic_total_cn_dict[classification]
                seg_allelic_cn[chrom][(start, end)] = basic_allelic_cn_dict[classification]
                seg_ccf[chrom][(start, end)] = sample_ccf_dict

    return seg_assignments, seg_total_cn, seg_allelic_cn, seg_ccf

In [10]:
def plot_hets(seg_assignments, hets_df, fn):
    plt.figure(figsize=(8, 6))
    ax = plt.gca()
    base_start = 0
    patch_color = 'white'
    chrom_ticks = []
    chroms = sorted(seg_assignments, key=lambda k: int(k) if k.isnumeric() else ord(k))

    for chrom in chroms:
        tree = seg_assignments[chrom]
        for seg in tree:
            hets = hets_df.loc[
                (hets_df['CONTIG'].astype(str) == chrom) & hets_df['POSITION'].astype(int).between(seg.begin, seg.end)]
            afs = hets['ALT_COUNT'] / (hets['REF_COUNT'] + hets['ALT_COUNT'])
            color = cluster_color_map[seg.data]
            ax.scatter(hets['POSITION'] + base_start, afs, color=color)
        p = patches.Rectangle((base_start, -10), CSIZE[chrom], 100, fill=True, facecolor=patch_color, edgecolor=None,
                              alpha=.1)  # Background
        ax.add_patch(p)
        patch_color = 'gray' if patch_color == 'white' else 'white'
        chrom_ticks.append(base_start + CSIZE[chrom] / 2)
        base_start += CSIZE[chrom]
    ax.set_xticks(chrom_ticks)
    ax.set_xticklabels(chroms, fontsize=6)
    ax.set_xlim(0, base_start)
    ax.set_xlabel("Chromosome")
    ax.set_ylabel("Allele Fraction")
    ax.set_ylim(0, 1)
    plt.setp(ax.spines.values(), color='gray', lw=.5)
    plt.setp([ax.get_xticklines(), ax.get_yticklines()], color='gray', lw=.5)
    plt.savefig(fn)
    plt.close()

In [11]:
def plot_absolute(sample_name, purity, ploidy, seg_assignments, contig_trees, seg, fn):
    plt.figure(figsize=(8, 6))
    ax = plt.gca()
    base_start = 0
    chrom_ticks = []
    patch_color = 'white'
    chroms = sorted(contig_trees, key=lambda k: int(k) if k.isnumeric() else ord(k))
    for chrom in chroms:
        contig_tree = contig_trees[chrom]
        assign_tree = seg_assignments[chrom]
        for contig_seg in contig_tree:
            cr1, cr2 = contig_seg.data[sample_name]
            for assign_seg in assign_tree[contig_seg.begin:contig_seg.end]:
                start = max(assign_seg.begin, contig_seg.begin)
                end = min(assign_seg.end, contig_seg.end)
#                 if chrom == '1' and cr2 > 4:
#                     print('extend')
#                     start -= 40000
#                     end += 40000
                color = cluster_color_map[assign_seg.data]
                ax.hlines(cr1, start + base_start, end + base_start, color=color, lw=5)
                ax.hlines(cr2, start + base_start, end + base_start, color=color, lw=5)
        p = patches.Rectangle((base_start, -10), CSIZE[chrom], 100, fill=True, facecolor=patch_color, edgecolor=None,
                              alpha=.1)  # Background
        ax.add_patch(p)
        patch_color = 'gray' if patch_color == 'white' else 'white'
        chrom_ticks.append(base_start + CSIZE[chrom] / 2)
        base_start += CSIZE[chrom]
    cr_diff = ((seg['mu.minor'] * seg['length']).sum() + (seg['mu.major'] * seg['length']).sum()) / \
              (ploidy * seg['length'].sum() - (1 - 1 / purity) * seg['length'].sum() * 2)
    c0 = (cr_diff / purity) - cr_diff
    yticks = np.arange(c0, c0 + cr_diff * 2.5, cr_diff)

    ax.set_xticks(chrom_ticks)
    ax.set_xticklabels(chroms, fontsize=6)
    ax.set_xlim(0, base_start)
    ax.set_yticks(yticks)
    ax.set_yticklabels(range(3), fontsize=16)
    ax.set_ylim(yticks[0] - cr_diff, yticks[-1] + cr_diff)
    plt.setp(ax.spines.values(), color='gray',lw=.5)
    plt.setp([ax.get_xticklines(), ax.get_yticklines()], color='gray',lw=.5)
    plt.xlabel("Chromosome")
    plt.ylabel("Allelic Copy Ratio")
    plt.savefig(fn, facecolor = 'white')
    plt.close()

### Load files and create objects

In [ ]:
# Input files required
cluster_df = pd.read_csv(PUT CLUSTER CCFS FILE HERE, sep='\t')
samples = [PUT SAMPLE IDS HERE]
acs_dfs = [pd.read_csv(acs, sep='\t') for acs in PUT LIST OF ALLELICCAPSEG FILES HERE]
hets_dfs = [pd.read_csv(het, sep='\t') for het in PUT LIST OF HET FILES HERE]
pur = [PUT PURITIES OF SAMPLES HERE]
plo = [PUT PLOIDIES OF SAMPLES HERE]

sample_ccfs = get_sample_ccfs(cluster_df, samples)
contig_trees = make_contig_trees(acs_dfs, samples)
seg_assignments, seg_total_cn, seg_allelic_cn, seg_ccf = assign_segs(contig_trees, sample_ccfs, hets_dfs, pur, plo)

### Examples

In [27]:
# Get input files from Terra through dalmatian
import dalmatian

#input
# patient_list = ['CLL-CRC-0002-0042', 'CLL-CRC-0004','CLL-CRC-0005-0043','CLL-CRC-0006',
#                 'CLL-CRC-0007', 'CLL-CRC-0008', 'CLL-CRC-0009', 'CLL-CRC-0010', 'CLL-CRC-0011','CLL-CRC-0012', 
#                 'CLL-CRC-0014', 'CLL-CRC-0015', 'CLL-CRC-0016', 'CLL-CRC-0017', 'CLL-CRC-0018', 'CLL-CRC-0019', 
#                'CLL-CRC-0020', 'CLL-CRC-0021', 'CLL-CRC-0036', 'CLL-CRC-0037', 'CLL-CRC-0038','CLL-CRC-0039', 
#                'CLL-CRC-0040', 'CLL-CRC-0041', 'CLL-CRC-0045']

patient_list = ['CLL-CRC-0006',
                'CLL-CRC-0007', 'CLL-CRC-0008', 'CLL-CRC-0009', 'CLL-CRC-0010', 'CLL-CRC-0011','CLL-CRC-0012', 
                'CLL-CRC-0014', 'CLL-CRC-0015', 'CLL-CRC-0016', 'CLL-CRC-0017', 'CLL-CRC-0018', 'CLL-CRC-0019', 
               'CLL-CRC-0020', 'CLL-CRC-0021', 'CLL-CRC-0036', 'CLL-CRC-0037', 'CLL-CRC-0038','CLL-CRC-0039', 
               'CLL-CRC-0040', 'CLL-CRC-0041', 'CLL-CRC-0045']

workspace ='broad-firecloud-ibmwatson/Getz-IBM-Wu_Fludarabine_Resistance-Analysis'
wm = dalmatian.WorkspaceManager(workspace)

participants = wm.get_participants()


In [28]:
for patient in patient_list:
    
    # In the CRC cohort, the pair set name is the same as the patient
    pair_set = patient

    cluster_df = pd.read_csv(participants.loc[patient]['cluster_ccfs_Aug2'], sep='\t')
    pairs = wm.get_pairs_in_pair_set(pair_set)
    
    samples = [i for i,pair in pairs.iterrows()]
    pur = [float(i) for i in pairs.wxs_purity.values]
    plo =  [float(i) for i in pairs.wxs_ploidy.values]
    acs_files = [pair['alleliccapseg_tsv'] for i,pair in pairs.iterrows()]
    hets_files = [pair['gatk_het_ad_tumor'] for i,pair in pairs.iterrows()]
    
    
    acs_dfs = [pd.read_csv(acs, sep='\t') for acs in acs_files]
    hets_dfs = [pd.read_csv(het, sep='\t') for het in hets_files]
    sample_ccfs = get_sample_ccfs(cluster_df, samples)
    contig_trees = make_contig_trees(acs_dfs, samples)
    seg_assignments, seg_total_cn, seg_allelic_cn, seg_ccf = assign_segs(contig_trees, sample_ccfs, hets_dfs, pur, plo)


    for sample_name, purity, ploidy, seg in zip(samples, pur, plo, acs_dfs):
            plot_absolute(sample_name, purity, ploidy, seg_assignments, contig_trees, seg,
                          'CRC_cnv' + '/' + sample_name + '.segs.extended.png')

    with open(patient + '.seg_assignments.txt', 'w') as f:
        f.write('Chromosome\tStart.bp\tEnd.bp\tCluster_assignment\ttotal_CN\tallelic_CN\n')
        for chrom in seg_assignments:
            for seg in seg_assignments[chrom]:
                tcn = seg_total_cn[chrom][(seg.begin, seg.end)]
                acn = seg_allelic_cn[chrom][(seg.begin, seg.end)]
                f.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(chrom, str(seg.begin), str(seg.end), seg.data, tcn, acn))

/Users/lil/opt/miniconda3/envs/Fludarabine/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:645: RuntimeWarning: divide by zero encountered in _beta_pdf
  return _boost._beta_pdf(x, a, b)
/Users/lil/opt/miniconda3/envs/Fludarabine/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:645: RuntimeWarning: divide by zero encountered in _beta_pdf
  return _boost._beta_pdf(x, a, b)
/Users/lil/opt/miniconda3/envs/Fludarabine/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:645: RuntimeWarning: divide by zero encountered in _beta_pdf
  return _boost._beta_pdf(x, a, b)
/Users/lil/opt/miniconda3/envs/Fludarabine/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:645: RuntimeWarning: divide by zero encountered in _beta_pdf
  return _boost._beta_pdf(x, a, b)
/Users/lil/opt/miniconda3/envs/Fludarabine/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:645: RuntimeWarning: divide by zero encountered in _beta_pdf
  return _boost._beta_pdf(x, a

### Write seg assignment file

In [19]:
with open(patient + '.seg_assignments.txt', 'w') as f:
        f.write('Chromosome\tStart.bp\tEnd.bp\tCluster_assignment\ttotal_CN\tallelic_CN\n')
        for chrom in seg_assignments:
            for seg in seg_assignments[chrom]:
                tcn = seg_total_cn[chrom][(seg.begin, seg.end)]
                acn = seg_allelic_cn[chrom][(seg.begin, seg.end)]
                f.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(chrom, str(seg.begin), str(seg.end), seg.data, tcn, acn))

### Write seg CCF file

In [12]:

with open('{}.seg_ccf.txt'.format('JB-0126'), 'w') as f:
    f.write('Chromosome\tStart\tEnd\tcn_a1\tcn_a2\t{}\n'.format('\t'.join(s + '_CCF' for s in samples)))
    for chrom, seg_dict in seg_ccf.items():
        for (start, end), sample_ccfs in seg_dict.items():
            f.write('{}\t{}\t{}\t{}\t{}\t'.format(chrom, start, end, seg_allelic_cn[chrom][(start, end)][0], seg_allelic_cn[chrom][(start, end)][1]))
            f.write('\t'.join(str(sample_ccfs[s]) for s in samples) + '\n')